In [ ]:
from types import SimpleNamespace
import numpy as np
import deepdish as dd
import os
import pandas as pd
pd.set_option('display.max_columns', None)

# Parse data

In [ ]:
def to_df(fname_start, vals_columns, remove_keys, save, fname):
    # Load all data from deepdish and convert it to pd.DataFrame
    files = [f for f in os.listdir('./') if f.startswith(fname_start) and f.endswith('.h5')]
    print(files)
    df = pd.DataFrame()
    for f in files:
        x = dd.io.load(f)
        df1 = pd.DataFrame(x.pop('vals'), columns=vals_columns)
        df2 = pd.DataFrame(x.pop('current_phase'))
        df_new = pd.concat([df1, df2], axis=1)

        for key in remove_keys:
            x.pop(key)

        for dim in x.keys():
            if isinstance(x[dim], dict):
                dic = {key: val for key, val in x[dim].items() if val is not None}
                df_new = df_new.assign(**dic)
            else:
                df_new[dim] = x[dim]

        df = df.append(df_new, ignore_index=True)

    if save:
        df.reindex().to_hdf(fname, 'all_data', mode='w')
    return df

In [ ]:
fname_start = 'no_disorder_B_vs_I_c'
vals_columns = ['T', 'B_x', 'orbital', 'g', 'alpha', 'mu']
remove_keys = ['T_list', 'T', 'B_x', 'orbital_list', 'g_list', 'alpha_list', 'mu_list']
save = True
fname = 'test.hdf'
df = to_df(fname_start, vals_columns, remove_keys, save, fname)

In [ ]:
fname_start = 'current_as_function_of_gate'
vals_columns = ['B_x', 'Vs']
remove_keys = ['B_x', 'Vs']
save = True
fname = 'test_gate.hdf'
df = to_df(fname_start, vals_columns, remove_keys, save, fname)